# Importing the required libraries

In [1]:
from kiteconnect import KiteConnect
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os 
import time
from time import sleep 
import statsmodels.api as sm
from pyotp import TOTP
import pyotp
import base64
import binascii
from datetime import datetime
from urllib.parse import urlparse,parse_qs
import pandas as pd 
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
# import mplfinance as mpf
# import plotly.graph_objects as go
import seaborn as sns
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint
import sys
sys.path.append("../FinRL-Library")
import itertools
from backtesting import Strategy,Backtest
from backtesting.lib import crossover
import ta

c:\Users\rsurs\miniconda3\envs\algo\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


# Automate user-login to kite-connect via selenium

In [2]:
def get_curr_path(folder_name):
    curr_dir = os.getcwd()
    curr_path = os.path.join(curr_dir,folder_name)
    return curr_path

def get_credentials(curr_path,file_name):
    file_dir = os.path.join(curr_path,file_name)
    file = open(file_dir,'r').read().split()
    api_key = file[0]
    api_secret = file[1]
    user_name = file[2]
    pwd = file[3]
    totp_key = file[-1]
    return api_key,api_secret,user_name,pwd,totp_key

def auto_login(api_key,user_name,pwd,totp_key):
    kite = KiteConnect(api_key=api_key)
    service = Service(ChromeDriverManager().install())
    service.start()
    options = Options()
    options.to_capabilities()
    driver = webdriver.Remote(
        command_executor=service.service_url,
        options=options)
    driver.get(kite.login_url())
    driver.implicitly_wait(5)
    username = driver.find_element(By.XPATH, "//input[@type='text']")
    username.send_keys(user_name)
    password = driver.find_element(By.XPATH, "//input[@type='password']")
    password.send_keys(pwd)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    sleep(1)
    totp = driver.find_element(By.XPATH,"//input[@type='number']")
    totp_token = TOTP(totp_key)
    token = totp_token.now()
    totp.send_keys(token)
    driver.find_element(By.XPATH,"//button[@type = 'submit']").click()
    sleep(1)
    current_url = driver.current_url
    parsed_url = urlparse(current_url)
    query_params = parse_qs(parsed_url.query)
    request_token = query_params.get('request_token',[None])[0]
    with open('request_token.txt', 'w') as f:
        f.write(request_token)
    request_token = open('request_token.txt','r').read()
    driver.quit()
    return request_token
    

def generate_access_token(request_token,api_key,api_secret):
    request_token = open('request_token.txt','r').read()
    kite = KiteConnect(api_key=api_key)
    data = kite.generate_session(request_token=request_token,api_secret=api_secret)
    data
    with open('access_token.txt','w') as f:
        f.write(data['access_token'])
    access_token = open('access_token.txt','r').read()
    return access_token

In [3]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

# Setting up the access token to execute buy,sell orders

In [4]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [5]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

def fetchOHLCExtended(ticker, inception_date, interval):
    """Extracts historical data and outputs in the form of a DataFrame.
       inception_date string format - dd-mm-yyyy"""
    instrument = instrumentLookup(instrument_df, ticker)
    from_date = dt.datetime.strptime(inception_date, '%d-%m-%Y')
    data = pd.DataFrame()  # Start with an empty DataFrame
    while True:
        if from_date.date() >= (dt.date.today() - dt.timedelta(100)):
            new_data = pd.DataFrame(kite.historical_data(instrument, from_date, dt.date.today(), interval))
            if not new_data.empty:
                if data.empty:
                    data = new_data
                else:
                    data = pd.concat([data, new_data], ignore_index=True)
            break
        else:
            to_date = from_date + dt.timedelta(100)
            new_data = pd.DataFrame(kite.historical_data(instrument, from_date, to_date, interval))
            if not new_data.empty:
                if data.empty:
                    data = new_data
                else:
                    data = pd.concat([data, new_data], ignore_index=True)
            from_date = to_date
    
    if not data.empty:
        data.set_index("date", inplace=True)
    return data

# Extracting 5 minute interval data of infosys stock

In [7]:
ohlc = fetchOHLC('INFY','5minute',30)

In [8]:
ohlc.shape

(1446, 5)

In [9]:
ohlc.head()

,open,high,low,close,volume
date,,,,,
2024-05-18 09:15:00+05:30,1445.00,1450.00,1444.30,1448.50,33712
2024-05-18 09:20:00+05:30,1448.50,1448.50,1447.05,1448.00,8477
2024-05-18 09:25:00+05:30,1448.10,1448.95,1447.95,1448.50,10257
2024-05-18 09:30:00+05:30,1448.60,1448.70,1447.25,1448.15,12140
2024-05-18 09:35:00+05:30,1448.15,1448.25,1447.15,1447.15,5802


# Computing the technical indicators 

<h3>Computing Exponential Moving Average</h3>
<ol>
    <li>There are three steps to calculate the EMA:</li>
    <li>Calculate the Simple Moving Average (SMA)</li>
    <li>Compute Multiplier = (2 / (n + 1))</li>
    <li>For the first EMA, use the SMA of the previous day. For subsequent EMAs, use:
        <br>EMA = {Close - EMA(previous day)} x Multiplier + EMA(previous day)
    </li>
</ol>

<h3>Procedure to Compute MACD</h3>
<ol>
    <li>MACD Line: (12-day EMA - 26-day EMA)</li>
    <li>Signal Line: 9-day EMA of the MACD Line</li>
    <li>MACD Histogram: MACD Line - Signal Line</li>
</ol>


In [10]:
temp_df = ohlc.copy()
temp_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-18 09:15:00+05:30,1445.00,1450.00,1444.30,1448.50,33712
2024-05-18 09:20:00+05:30,1448.50,1448.50,1447.05,1448.00,8477
2024-05-18 09:25:00+05:30,1448.10,1448.95,1447.95,1448.50,10257
2024-05-18 09:30:00+05:30,1448.60,1448.70,1447.25,1448.15,12140
2024-05-18 09:35:00+05:30,1448.15,1448.25,1447.15,1447.15,5802


In [11]:
temp_df.to_csv('dummy.csv') 

In [12]:
temp_df.shape

(1446, 5)

In [13]:
def compute_ema(df,n,column_name):
    df[column_name] = np.nan
    df.loc[df.index[n-1], column_name] = df['close'][:n].mean()
    multiplier = 2 / (n + 1)
    for i in range(n,len(df)):
        df.loc[df.index[i],column_name] = (df.loc[df.index[i],'close'] - df.loc[df.index[i - 1],column_name]) * multiplier + df.loc[df.index[i - 1],column_name]
    return df

In [14]:
temp_df = compute_ema(temp_df,12,'fast_ma')
temp_df = compute_ema(temp_df,26,'slow_ma')
temp_df['macd'] = temp_df['fast_ma'] - temp_df['slow_ma']

In [15]:
temp_df 

,open,high,low,close,volume,fast_ma,slow_ma,macd
date,,,,,,,,
2024-05-18 09:15:00+05:30,1445.00,1450.00,1444.30,1448.50,33712,NaN,NaN,NaN
2024-05-18 09:20:00+05:30,1448.50,1448.50,1447.05,1448.00,8477,NaN,NaN,NaN
2024-05-18 09:25:00+05:30,1448.10,1448.95,1447.95,1448.50,10257,NaN,NaN,NaN
2024-05-18 09:30:00+05:30,1448.60,1448.70,1447.25,1448.15,12140,NaN,NaN,NaN
2024-05-18 09:35:00+05:30,1448.15,1448.25,1447.15,1447.15,5802,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252


In [16]:
def compute_signal(df, n):
    df['signal'] = np.nan
    
    first_valid_index = df['macd'].first_valid_index()
    if first_valid_index is None:
        raise ValueError("MACD column contains only NaN values.")
    
    start_index = df.index.get_loc(first_valid_index) + n - 1
    if start_index < len(df):
        df.loc[df.index[start_index], 'signal'] = df['macd'].iloc[start_index - n + 1:start_index + 1].mean()
    multiplier = 2 / (n + 1)
    for i in range(start_index + 1, len(df)):
        df.loc[df.index[i], 'signal'] = ((df.loc[df.index[i], 'macd'] - df.loc[df.index[i - 1], 'signal']) 
                                          * multiplier) + df.loc[df.index[i - 1], 'signal']   
    return df

In [17]:
temp_df = compute_signal(temp_df,9)
temp_df

,open,high,low,close,volume,fast_ma,slow_ma,macd,signal
date,,,,,,,,,
2024-05-18 09:15:00+05:30,1445.00,1450.00,1444.30,1448.50,33712,NaN,NaN,NaN,NaN
2024-05-18 09:20:00+05:30,1448.50,1448.50,1447.05,1448.00,8477,NaN,NaN,NaN,NaN
2024-05-18 09:25:00+05:30,1448.10,1448.95,1447.95,1448.50,10257,NaN,NaN,NaN,NaN
2024-05-18 09:30:00+05:30,1448.60,1448.70,1447.25,1448.15,12140,NaN,NaN,NaN,NaN
2024-05-18 09:35:00+05:30,1448.15,1448.25,1447.15,1447.15,5802,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326,-0.365945
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574,-0.322671
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252,-0.294587


In [18]:
temp_df['macd_histogram'] = temp_df['macd'] - temp_df['signal']
temp_df

,open,high,low,close,volume,fast_ma,slow_ma,macd,signal,macd_histogram
date,,,,,,,,,,
2024-05-18 09:15:00+05:30,1445.00,1450.00,1444.30,1448.50,33712,NaN,NaN,NaN,NaN,NaN
2024-05-18 09:20:00+05:30,1448.50,1448.50,1447.05,1448.00,8477,NaN,NaN,NaN,NaN,NaN
2024-05-18 09:25:00+05:30,1448.10,1448.95,1447.95,1448.50,10257,NaN,NaN,NaN,NaN,NaN
2024-05-18 09:30:00+05:30,1448.60,1448.70,1447.25,1448.15,12140,NaN,NaN,NaN,NaN,NaN
2024-05-18 09:35:00+05:30,1448.15,1448.25,1447.15,1447.15,5802,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326,-0.365945,0.168619
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574,-0.322671,0.173096
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252,-0.294587,0.112335


In [19]:
temp_df.to_csv('macd.csv')

<h3>Procedure for calculating the rsi</h3>
<ol>
  <li><b>Formula for RSI</b>:
    <br>RSI = 100 - 100 / (1 + RS)
  </li>
  <li><b>Calculation of RS (Relative Strength)</b>:
    <br>RS = Average Gain of n days UP / Average Loss of n days DOWN
  </li>
  <li><b>Determine Change</b>:
    <br>change = change(close)
  </li>
  <li><b>Calculate Gain</b>:
    <br>gain = change >= 0 ? change : 0.0
  </li>
  <li><b>Calculate Loss</b>:
    <br>loss = change < 0 ? (-1) * change : 0.0
  </li>
  <li><b>Compute Average Gain</b>:
    <br>avgGain = rma(gain, 14)
    <br>- rma denotes the running moving average.
  </li>
  <li><b>Compute Average Loss</b>:
    <br>avgLoss = rma(loss, 14)
  </li>
  <li><b>Calculate RS</b>:
    <br>rs = avgGain / avgLoss
  </li>
  <li><b>Calculate RSI</b>:
    <br>rsi = 100 - (100 / (1 + rs))
  </li>
  <li><b>RSI Equivalence</b>:
    <br>The calculated RSI value, as shown above, is exactly equal to:
    <br>rsi(close, 14)
  </li>
</ol>


In [20]:
def compute_rsi(df,n):
    df['close_diff'] = df['close'].diff().dropna()
    # df.dropna(inplace=True)
    df['gain'] = df['close_diff'].apply(lambda x: x if x > 0 else 0)
    df['loss'] = df['close_diff'].apply(lambda x: -x if x < 0 else 0)
    alpha = 1/n
    
    for i in range(n,len(df)):
        if i == n:
            df.loc[df.index[i],'avg_gain'] = df['gain'].rolling(window=n,min_periods=n).mean().iloc[n-1]
            df.loc[df.index[i],'avg_loss'] = df['loss'].rolling(window=n,min_periods=n).mean().iloc[n-1]
        else:
            df.loc[df.index[i],'avg_gain'] = (alpha * df.loc[df.index[i],'gain']) + (1 - alpha)*(df.loc[df.index[i-1],'avg_gain'])
            df.loc[df.index[i],'avg_loss'] = (alpha * df.loc[df.index[i],'loss']) + (1 - alpha)*(df.loc[df.index[i-1],'avg_loss'])
    
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['rsi'] = 100 - (100/(1 + df['rs']))
    return df

In [21]:
copy_df = temp_df.copy()

In [22]:
copy_df = compute_rsi(copy_df,14)

In [23]:
copy_df

,open,high,low,close,volume,fast_ma,slow_ma,macd,signal,macd_histogram,close_diff,gain,loss,avg_gain,avg_loss,rs,rsi
date,,,,,,,,,,,,,,,,,
2024-05-18 09:15:00+05:30,1445.00,1450.00,1444.30,1448.50,33712,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
2024-05-18 09:20:00+05:30,1448.50,1448.50,1447.05,1448.00,8477,NaN,NaN,NaN,NaN,NaN,-0.50,0.00,0.50,NaN,NaN,NaN,NaN
2024-05-18 09:25:00+05:30,1448.10,1448.95,1447.95,1448.50,10257,NaN,NaN,NaN,NaN,NaN,0.50,0.50,0.00,NaN,NaN,NaN,NaN
2024-05-18 09:30:00+05:30,1448.60,1448.70,1447.25,1448.15,12140,NaN,NaN,NaN,NaN,NaN,-0.35,0.00,0.35,NaN,NaN,NaN,NaN
2024-05-18 09:35:00+05:30,1448.15,1448.25,1447.15,1447.15,5802,NaN,NaN,NaN,NaN,NaN,-1.00,0.00,1.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326,-0.365945,0.168619,0.10,0.10,0.00,0.187571,0.176511,1.062660,51.518909
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574,-0.322671,0.173096,0.05,0.05,0.00,0.177744,0.163903,1.084450,52.025709
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252,-0.294587,0.112335,-0.90,0.00,0.90,0.165048,0.216481,0.762414,43.259652


In [24]:
# copy_df.isna().sum()

In [25]:
copy_df.dropna(inplace=True)

In [26]:
# len(copy_df)

In [27]:
copy_df

,open,high,low,close,volume,fast_ma,slow_ma,macd,signal,macd_histogram,close_diff,gain,loss,avg_gain,avg_loss,rs,rsi
date,,,,,,,,,,,,,,,,,
2024-05-21 10:15:00+05:30,1430.90,1431.50,1429.15,1430.65,63418,1432.528922,1437.275530,-4.746608,-4.991216,0.244608,-0.20,0.00,0.2,0.222212,0.716078,0.310318,23.682638
2024-05-21 10:20:00+05:30,1430.65,1431.00,1430.00,1430.90,43714,1432.278319,1436.803269,-4.524950,-4.897963,0.373013,0.25,0.25,0.0,0.224197,0.664930,0.337174,25.215389
2024-05-21 10:25:00+05:30,1430.90,1431.15,1430.00,1430.60,32261,1432.020116,1436.343767,-4.323652,-4.783101,0.459449,-0.30,0.00,0.3,0.208183,0.638864,0.325864,24.577491
2024-05-21 10:30:00+05:30,1430.35,1432.00,1430.20,1431.60,66979,1431.955483,1435.992377,-4.036895,-4.633860,0.596965,1.00,1.00,0.0,0.264741,0.593230,0.446270,30.856629
2024-05-21 10:35:00+05:30,1431.35,1433.40,1431.10,1433.20,52779,1432.146947,1435.785534,-3.638588,-4.434805,0.796218,1.60,1.60,0.0,0.360117,0.550857,0.653739,39.530974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326,-0.365945,0.168619,0.10,0.10,0.0,0.187571,0.176511,1.062660,51.518909
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574,-0.322671,0.173096,0.05,0.05,0.0,0.177744,0.163903,1.084450,52.025709
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252,-0.294587,0.112335,-0.90,0.00,0.9,0.165048,0.216481,0.762414,43.259652


# Target profit and stop loss 

In [28]:
def compute_targets(df,risk,reward):
    close_price = df['close'].iloc[-1]
    stop_loss = close_price - risk 
    take_profit = close_price + reward 
    return stop_loss,take_profit

In [29]:
sl,tp = compute_targets(copy_df,2,4)
print(sl,tp)

1486.6 1492.6


In [30]:
backtest_df = copy_df.copy()

# Backtesting with backtesting.py

In [31]:
backtest_df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume',
    'macd':'MACD',
    'signal': 'MACD_SIGNAL',
    'rsi':'RSI'
}, inplace=True)

In [32]:
backtest_df

,Open,High,Low,Close,Volume,fast_ma,slow_ma,MACD,MACD_SIGNAL,macd_histogram,close_diff,gain,loss,avg_gain,avg_loss,rs,RSI
date,,,,,,,,,,,,,,,,,
2024-05-21 10:15:00+05:30,1430.90,1431.50,1429.15,1430.65,63418,1432.528922,1437.275530,-4.746608,-4.991216,0.244608,-0.20,0.00,0.2,0.222212,0.716078,0.310318,23.682638
2024-05-21 10:20:00+05:30,1430.65,1431.00,1430.00,1430.90,43714,1432.278319,1436.803269,-4.524950,-4.897963,0.373013,0.25,0.25,0.0,0.224197,0.664930,0.337174,25.215389
2024-05-21 10:25:00+05:30,1430.90,1431.15,1430.00,1430.60,32261,1432.020116,1436.343767,-4.323652,-4.783101,0.459449,-0.30,0.00,0.3,0.208183,0.638864,0.325864,24.577491
2024-05-21 10:30:00+05:30,1430.35,1432.00,1430.20,1431.60,66979,1431.955483,1435.992377,-4.036895,-4.633860,0.596965,1.00,1.00,0.0,0.264741,0.593230,0.446270,30.856629
2024-05-21 10:35:00+05:30,1431.35,1433.40,1431.10,1433.20,52779,1432.146947,1435.785534,-3.638588,-4.434805,0.796218,1.60,1.60,0.0,0.360117,0.550857,0.653739,39.530974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326,-0.365945,0.168619,0.10,0.10,0.0,0.187571,0.176511,1.062660,51.518909
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574,-0.322671,0.173096,0.05,0.05,0.0,0.177744,0.163903,1.084450,52.025709
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252,-0.294587,0.112335,-0.90,0.00,0.9,0.165048,0.216481,0.762414,43.259652


In [33]:
def generate_buy_sell_signals(df):
    df['Signal'] = np.where((df['MACD'] > df['MACD_SIGNAL']) & (df['RSI'] < 30), 1,
                          np.where((df['MACD'] < df['MACD_SIGNAL']) & (df['RSI'] > 70), -1, 0))
    return df

In [34]:
backtest_df = generate_buy_sell_signals(backtest_df)
backtest_df

,Open,High,Low,Close,Volume,fast_ma,slow_ma,MACD,MACD_SIGNAL,macd_histogram,close_diff,gain,loss,avg_gain,avg_loss,rs,RSI,Signal
date,,,,,,,,,,,,,,,,,,
2024-05-21 10:15:00+05:30,1430.90,1431.50,1429.15,1430.65,63418,1432.528922,1437.275530,-4.746608,-4.991216,0.244608,-0.20,0.00,0.2,0.222212,0.716078,0.310318,23.682638,1
2024-05-21 10:20:00+05:30,1430.65,1431.00,1430.00,1430.90,43714,1432.278319,1436.803269,-4.524950,-4.897963,0.373013,0.25,0.25,0.0,0.224197,0.664930,0.337174,25.215389,1
2024-05-21 10:25:00+05:30,1430.90,1431.15,1430.00,1430.60,32261,1432.020116,1436.343767,-4.323652,-4.783101,0.459449,-0.30,0.00,0.3,0.208183,0.638864,0.325864,24.577491,1
2024-05-21 10:30:00+05:30,1430.35,1432.00,1430.20,1431.60,66979,1431.955483,1435.992377,-4.036895,-4.633860,0.596965,1.00,1.00,0.0,0.264741,0.593230,0.446270,30.856629,0
2024-05-21 10:35:00+05:30,1431.35,1433.40,1431.10,1433.20,52779,1432.146947,1435.785534,-3.638588,-4.434805,0.796218,1.60,1.60,0.0,0.360117,0.550857,0.653739,39.530974,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 15:05:00+05:30,1488.50,1489.25,1488.35,1488.95,89282,1488.584629,1488.781955,-0.197326,-0.365945,0.168619,0.10,0.10,0.0,0.187571,0.176511,1.062660,51.518909,0
2024-06-14 15:10:00+05:30,1489.00,1489.65,1488.50,1489.00,91719,1488.648532,1488.798106,-0.149574,-0.322671,0.173096,0.05,0.05,0.0,0.177744,0.163903,1.084450,52.025709,0
2024-06-14 15:15:00+05:30,1488.95,1489.00,1488.00,1488.10,120812,1488.564142,1488.746395,-0.182252,-0.294587,0.112335,-0.90,0.00,0.9,0.165048,0.216481,0.762414,43.259652,0


In [35]:
print(len(backtest_df[backtest_df['Signal'] == 1]))
print(len(backtest_df[backtest_df['Signal'] == -1]))

11
27


In [36]:
# backtest_df = backtest_df.resample('1D').agg({
#     'Open': 'first',
#     'High': 'max',
#     'Low': 'min',
#     'Close': 'last',
#     'Volume': 'sum',
#     'MACD': 'last',
#     'MACD_SIGNAL': 'last',
#     'RSI': 'last',
#     'Signal': 'last'
# }).dropna()



In [37]:
backtest_df.to_csv('backtest.csv')

In [38]:
from backtesting import Strategy,Backtest

class MACD_RSI_Strategy(Strategy):

    def init(self):
        pass

    def next(self):
        rsi = self.data.RSI[-1]
        macd = self.data.MACD[-1]
        macd_signal = self.data.MACD_SIGNAL[-1]
        curr_signal = self.data.Signal[-1]

        if curr_signal == 1:
            if not self.position:
                self.buy()
        elif curr_signal == -1:
            if self.position:
                self.position.close()

        

In [39]:
bt = Backtest(backtest_df,MACD_RSI_Strategy,cash=10000,commission=0.002)
stats = bt.run()
bt.plot()
print(stats)

c:\Users\rsurs\miniconda3\envs\algo\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\rsurs\miniconda3\envs\algo\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\rsurs\miniconda3\envs\algo\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
c:\Users\rsurs\miniconda3\envs\algo\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\rsurs\miniconda3\envs\algo\lib\site-packages\backtesting

Start                     2024-05-21 10:15...
End                       2024-06-14 15:25...
Duration                     24 days 05:10:00
Exposure Time [%]                    74.87615
Equity Final [$]                   10512.1913
Equity Peak [$]                     10662.006
Return [%]                           5.121913
Buy & Hold Return [%]                4.050606
Return (Ann.) [%]                   91.224812
Volatility (Ann.) [%]               40.222624
Sharpe Ratio                         2.267998
Sortino Ratio                        9.541479
Calmar Ratio                        14.024365
Max. Drawdown [%]                   -6.504737
Avg. Drawdown [%]                   -0.427352
Max. Drawdown Duration       14 days 00:45:00
Avg. Drawdown Duration        0 days 20:39:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                       3.679397
Worst Trade [%]                     -1.004094
Avg. Trade [%]                    

In [40]:
# copy_df.to_csv('backtest.csv')

In [41]:
# backtest_df = copy_df.copy()

In [42]:
# backtest_df.columns

In [43]:
# backtest_df.drop(['fast_ma', 'slow_ma', 'close_diff', 'gain', 'loss', 'avg_gain',
#        'avg_loss', 'rs'],axis=1,inplace=True)

In [44]:
# backtest_df = backtest_df.rename(columns={
#     'open': 'Open',
#     'high': 'High',
#     'low': 'Low',
#     'close': 'Close',
#     'volume': 'Volume',
#     'macd': 'MACD',
#     'signal': 'SIGNAL',
#     'macd_histogram': 'MACD_HIST',
#     'rsi': 'RSI'
# })


In [45]:
# backtest_df.columns

In [46]:
# backtest_df

In [47]:
# class MacdRsi(Strategy):
#     stop_loss = 0.05  
#     take_profit = 0.1

#     def init(self):
#         pass
    
#     def next(self):
#         # Access precomputed RSI, MACD, and MACD_signal
#         rsi = self.data.RSI[-1]
#         macd = self.data.MACD[-1]
#         macd_signal = self.data.SIGNAL[-1]

#         # Buy Signal: RSI < 30 and MACD > MACD_signal
#         if rsi < 30 and macd > macd_signal:
#             self.buy(sl=self.data.Close[-1] * (1 - self.stop_loss),
#                      tp=self.data.Close[-1] * (1 + self.take_profit))
        
#         # Sell Signal: RSI > 70 and MACD_signal > MACD
#         elif rsi > 70 and macd_signal > macd:
#             self.sell(sl=self.data.Close[-1] * (1 + self.stop_loss),
#                       tp=self.data.Close[-1] * (1 - self.take_profit))


In [48]:
# backtest_df_resampled = backtest_df.resample('D').agg({
#     'Open': 'first',
#     'High': 'max',
#     'Low': 'min',
#     'Close': 'last',
#     'Volume': 'sum',
#     'RSI': 'last',
#     'MACD': 'last',
#     'SIGNAL': 'last'
# }).dropna()


In [49]:
# bt = Backtest(backtest_df, MacdRsi, cash=10000, commission=.002)
# stats = bt.run()
# bt.plot()

# # Display backtest statistics
# print(stats)


In [50]:
# print("Checking column lengths:")
# for column in backtest_df.columns:
#     print(f"{column}: {len(backtest_df[column])}")

# # Ensure that all columns have the same number of entries
# print("\nChecking for missing values:")
# print(backtest_df.isna().sum())

# print("\nDataFrame columns:")
# print(backtest_df.columns)

# print("\nDataFrame head:")
# print(backtest_df.head())


# Trend directions

In [51]:
# def check_trend(df):
#     df['trend'] = np.NaN
#     for i in range(len(df)):
#         # Check for bullish trend
#         if (df['macd'].iloc[i] > df['signal'].iloc[i]) and (df['rsi'].iloc[i] < 30):
#             df.loc[df.index[i], 'trend'] = "bullish"
#         # Check for bearish trend
#         elif (df['macd'].iloc[i] < df['signal'].iloc[i]) and (df['rsi'].iloc[i] > 70):
#             df.loc[df.index[i], 'trend'] = "bearish"
#     return df

In [52]:
# new_df = copy_df.copy()

In [53]:
# new_df = check_trend(new_df)
# new_df


In [54]:
# print('bearish',len(new_df[new_df['trend'] == 'bearish']))
# print('bullish',len(new_df[new_df['trend'] == 'bullish']))


In [55]:
# new_df.to_csv('trends.csv')

In [56]:
# new_df.dropna(inplace=True)

In [57]:
# new_df

# Placing Buy and Sell orders

In [58]:
# def placeSLOrder(symbol,buy_sell,quantity,sl_price,take_profit):    
#     # Place an intraday stop loss order on NSE - handles market orders converted to limit orders
#     if buy_sell == "buy":
#         t_type = kite.TRANSACTION_TYPE_BUY
#         t_type_sl = kite.TRANSACTION_TYPE_SELL
#         t_type_target = kite.TRANSACTION_TYPE_SELL

#     elif buy_sell == "sell":
#         t_type=kite.TRANSACTION_TYPE_SELL
#         t_type_sl=kite.TRANSACTION_TYPE_BUY

#     market_order = kite.place_order(tradingsymbol=symbol,
#                     exchange=kite.EXCHANGE_NSE,
#                     transaction_type=t_type,
#                     quantity=quantity,
#                     order_type=kite.ORDER_TYPE_MARKET,
#                     product=kite.PRODUCT_CNC,
#                     variety=kite.VARIETY_REGULAR)
    
#     target_order = kite.place_order(tradingsymbol=symbol,
#                  exchange=kite.EXCHANGE_NSE,
#                  transaction_type=t_type_target,
#                  quantity=quantity,
#                  order_type=kite.ORDER_TYPE_LIMIT,
#                  price=take_profit,  # Define this as your take-profit price
#                  product=kite.PRODUCT_CNC,
#                  variety=kite.VARIETY_REGULAR)

#     a = 0
#     while a < 10:
#         try:
#             order_list = kite.orders()
#             break
#         except:
#             print("can't get orders..retrying")
#             a+=1
#     for order in order_list:
#         if order["order_id"]==market_order:
#             if order["status"]=="COMPLETE":
#                 kite.place_order(tradingsymbol=symbol,
#                                 exchange=kite.EXCHANGE_NSE,
#                                 transaction_type=t_type_sl,
#                                 quantity=quantity,
#                                 order_type=kite.ORDER_TYPE_SL,
#                                 price=sl_price,
#                                 trigger_price = sl_price,
#                                 product=kite.PRODUCT_CNC,
#                                 variety=kite.VARIETY_REGULAR)
#             else:
#                 kite.cancel_order(order_id=market_order,variety=kite.VARIETY_REGULAR)

In [59]:
# def generate_signals(df):
#     df = df.copy()
#     buy_signals = []
#     sell_signals = []
#     for i in range(len(df)):
#         if df['trend'].iloc[i] == 'bullish':
#             buy_signals.append(df['close'].iloc[i])
#             sell_signals.append(np.nan)
#         elif df['trend'].iloc[i] == 'bearish':
#             buy_signals.append(np.nan)
#             sell_signals.append(df['close'].iloc[i])
#         else:
#             buy_signals.append(np.nan)
#             sell_signals.append(np.nan)
#     df['buy_signal'] = buy_signals
#     df['sell_signal'] = sell_signals
#     return df

In [60]:
# signals_df = generate_signals(new_df) 

In [61]:
# len(signals_df['buy_signal']),len(signals_df['sell_signal'])

In [62]:
# buy_sell_df = signals_df.copy()

In [63]:
#buy_sell_df.dropna(inplace=True)

In [64]:
# buy_sell_df.fillna(0,inplace=True)

In [65]:
# print(buy_sell_df['buy_signal'].isna().value_counts())

In [66]:
# buy_sell_df.to_csv('buy_sell.csv')

In [67]:
# buy_sell_df